In [ ]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
import tensorflow as tf
tf.config.experimental.set_visible_devices([], "GPU")

from colabtools import adhoc_import
import importlib
from simulation_research.diffusion import ode_datasets
from simulation_research.diffusion import diffusion_unet
from simulation_research.diffusion import samplers
from simulation_research.diffusion import train
importlib.reload(ode_datasets)
importlib.reload(diffusion_unet)
importlib.reload(samplers)
importlib.reload(train)

import matplotlib.pyplot as plt
from matplotlib import rc
rc('animation', html='jshtml')
import jax.numpy as jnp
import numpy as np
import jax

In [ ]:
dt = .5
bs = 400
ds = ode_datasets.NPendulum(N=4000+bs,n=2,dt=dt)


thetas,vs = ode_datasets.unpack(ds.Zs[bs:])
test_x = ode_datasets.unpack(ds.Zs[:bs])[0]
#thetas /=thetas.std()
#thetas = jax.random.normal(jax.random.PRNGKey(38),thetas.shape)
dataset = tf.data.Dataset.from_tensor_slices(thetas)

dataiter = dataset.shuffle(len(dataset)).batch(bs).as_numpy_iterator

In [ ]:

## Check that the computed velocities from $v=M(q)^{-1}p$ match those from finite differences.


import matplotlib.pyplot as plt
vfd = -(thetas[:,:-2]-thetas[:,2:])/(2*dt)
plt.plot(ds.T_long[1:-1],vs[0,1:-1,0])
plt.plot(ds.T_long[1:-1],vfd[0,:,0])
plt.xlabel('Time t')
plt.ylabel(r'$\dot\theta$')
plt.legend(['v state','v finite diff'])

In [ ]:
## Examine the state over time

import matplotlib.pyplot as plt
plt.plot(ds.T_long,thetas[0,:,0])
plt.plot(ds.T_long,thetas[0,:,1])
plt.xlabel('Time t')
plt.ylabel(r'State')
plt.legend([r'$\theta_0$',r'$\theta_1$'])

In [ ]:
x = test_x#next(dataiter())
t = np.random.rand(x.shape[0])
model = diffusion_unet.UNet(diffusion_unet.unet_64_config(out_dim=x.shape[-1],base_channels=24))

In [ ]:
#%env XLA_FLAGS=--xla_gpu_strict_conv_algorithm_picker=false
difftype='VE'#@param ['VP','VE','SubVP','Test']
diff = {'VP':train.VariancePreserving,'VE':train.VarianceExploding,
        'SubVP':train.SubVariancePreserving,'Test':train.Test}[difftype]
epochs = 400#@param {'type':'integer'}
score_fn = train.train_diffusion(model,dataiter,epochs,diffusion=diff,lr=1e-3)

In [ ]:
import jax
key= jax.random.PRNGKey(38)
sample_traj = samplers.stochastic_sample(diff,score_fn,key,x[:30].shape,N=1000,traj=True)

In [ ]:
sample_traj.shape

In [ ]:
xt = sample_traj[:,:1]
N=1000
timesteps = (.5+np.arange(N)[::-1])/N
ts = (timesteps[:-1])
J= jax.jacfwd(score_fn)(xt[0].reshape(-1),ts[0])
tt = ts[::200]
Js = jax.vmap(jax.jacfwd(score_fn))(xt.reshape(xt.shape[0],-1)[::200],tt)*(diff.sigma(tt)**2)[:,None,None]
w,v = jnp.linalg.eigh(Js)

In [ ]:
for i,t in enumerate(ts[::200]):
  plt.plot(w[i],label=str(t))
plt.legend()
plt.ylabel(r'$\lambda_i$')
plt.xlabel(r'$i$')
#plt.ylim(1e-3,2)

In [ ]:
for i,t in enumerate(ts[::200]):
  plt.plot(1+w[i],label=str(t))
plt.yscale('log')
plt.legend()
plt.ylabel(r'$(1+\lambda_i)$')
plt.xlabel(r'$i$')
#plt.ylim(1e-3,2)

In [ ]:
from jax import grad,jit
condition_amount = 13# @param {type:"slider", min:0, max:50, step:1}
mb = x[:30,:]


def inpainting_scores(diffusion,scorefn,observed_values,slc):
  b,n,c = observed_values.shape
  def conditioned_scores(xt,t):
    unflat_xt = xt.reshape(b,-1,c)

    observed_score = diffusion.noise_score(unflat_xt[:,slc],observed_values,t)
    unobserved_score = scorefn(xt,t).reshape(b,-1,c)
    combined_score = unobserved_score.at[:,slc].set(observed_score)
    return combined_score.reshape(-1)
  return conditioned_scores

def inpainting_scores2(diffusion,scorefn,observed_values,slc):
  b,n,c = observed_values.shape
  def conditioned_scores(xt,t):
    unflat_xt = xt.reshape(b,-1,c)

    observed_score = diffusion.noise_score(unflat_xt[:,slc],observed_values,t)
    unobserved_score = scorefn(xt,t).reshape(b,-1,c)
    def constraint(xt):
      one_step_xhat = (xt+diffusion.sigma(t)**2*scorefn(xt,t))/diffusion.scale(t)
      return jnp.sum((one_step_xhat.reshape(b,-1,c)[:,slc]-observed_values)**2)
    #unobserved_score -= grad(constraint)(xt).reshape(unflat_xt.shape)*10/(diff.g2(t)/2)
    unobserved_score -= grad(constraint)(xt).reshape(unflat_xt.shape)*10*diff.scale(t)**2/diff.sigma(t)**2
    combined_score = unobserved_score.at[:,slc].set(observed_score)
    return combined_score.reshape(-1)
  return jit(conditioned_scores)

slc = slice(condition_amount)
conditioned_samples = samplers.stochastic_sample(diff,inpainting_scores2(diff,score_fn,mb[:,slc],slc),key,mb.shape,N=1000,traj=True)


In [ ]:
xt = conditioned_samples[:,:1]
N=1000
timesteps = (.5+np.arange(N)[::-1])/N
ts = (timesteps[:-1])
J= jax.jacfwd(score_fn)(xt[0].reshape(-1),ts[0])
tt = ts[::25]
Js = jax.vmap(jax.jacfwd(score_fn))(xt.reshape(xt.shape[0],-1)[::25],tt)*(diff.sigma(tt)**2)[:,None,None]
Js = Js[:,slc,slc]
w,v = jnp.linalg.eigh(Js)

In [ ]:
jnp.linalg.norm(Js-jnp.swapaxes(Js,-1,-2),axis=(-1,-2))/jnp.linalg.norm(Js,axis=(-1,-2))

In [ ]:
for i,t in enumerate(tt):
  plt.plot(w[i],label=str(t))
plt.legend()
plt.ylabel(r'$\lambda_i$')
plt.xlabel(r'$i$')
#plt.ylim(1e-3,2)

In [ ]:
plt.plot(tt,1+w[:,-1],label=r'$1+\lambda_0$')
plt.plot(tt,1-1/(1+1e0/diff.sigma(tt)**2),label=r'$1/\sqrt{\sigma_t}$')
plt.yscale('log')
plt.legend()

In [ ]:
for i,t in enumerate(tt):
  plt.plot(1/(1.05+w[i]),label=str(t))
plt.yscale('log')
plt.legend()
plt.ylabel(r'$(1+\lambda_i)$')
plt.xlabel(r'$i$')
#plt.ylim(1e-3,2)

In [ ]:
importlib.reload(samplers)
importlib.reload(train)
#samplers.probability_flow(diff,score_fn,x,1e-4,1.).std()

In [ ]:
import jax
key= jax.random.PRNGKey(38)
samplers.compute_nll(diff,score_fn,key,x).mean()

Sample generation

In [ ]:
stoch_samples = samplers.stochastic_sample(diff,score_fn,key,x[:30].shape,N=1000,traj=False)
sample_traj = samplers.stochastic_sample(diff,score_fn,key,x[:30].shape,N=1000,traj=True)
det_samples = samplers.sample(diff,score_fn,key,x[:30].shape)

In [ ]:
import matplotlib.pyplot as plt
i=13 #@param {type:"slider", min:0, max:30, step:1}
plt.plot(ds.T_long,sample_traj[400::100,i,:,-1].T,alpha=1/2)
plt.xlabel('Time t')
plt.ylabel(r'State')
plt.ylim(-2,2)
#plt.legend([r'GT',r'Model'])

In [ ]:
|from jax import vmap
n=sample_traj.shape[0]+1
ts = (.5+jnp.arange(n)[::-1])[:-1]/n
scores = vmap(score_fn)(sample_traj.reshape(len(ts),-1),ts).reshape(sample_traj.shape)
best_reconstructions = (sample_traj+diff.sigma(ts)[:,None,None,None]**2*scores)/diff.scale(ts)[:,None,None,None]

In [ ]:
import matplotlib.pyplot as plt

import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable

i=7 #@param {type:"slider", min:0, max:30, step:1}

cmap='inferno'


fig1 = plt.figure()
ax1 = fig1.add_subplot(111)
data = best_reconstructions[100::25,i,:,-1].T
ax1.plot(ds.T_long,data[:],alpha=.6,lw=2)
colors=list(mpl.cm.get_cmap(cmap)(np.linspace(0,1,len(ax1.lines))))
#colors = [colors(i) for i in np.linspace(0, 1,len(ax1.lines))]
for i,j in enumerate(ax1.lines):
    j.set_color(colors[i])
plt.xlabel('Time t')
plt.ylabel(r'State')
plt.ylim(-2,2)
divider = make_axes_locatable(plt.gca())
ax_cb = divider.new_horizontal(size="5%", pad=0.05)
norm = mpl.colors.Normalize(vmin=ts[100], vmax=ts[-25])    
cb1 = mpl.colorbar.ColorbarBase(ax_cb, cmap=mpl.cm.get_cmap(f'{cmap}_r'), orientation='vertical',norm=norm)
#cb1.ax.invert_yaxis()
cb1.set_label('diffusion time (0,1)')
plt.gcf().add_axes(ax_cb)

In [ ]:
from scipy.ndimage import correlate1d
i=7 #@param {type:"slider", min:0, max:30, step:1}
vs = -correlate1d(best_reconstructions,np.array([-1,0,1])/2/(ds.T[1]-ds.T[0]),axis=2)
print(vs.shape)
fig1 = plt.figure()
ax1 = fig1.add_subplot(111)
data = vs[100::25,i,:,-1].T
ax1.plot(ds.T_long,data[:],alpha=.6,lw=2)
colors=list(mpl.cm.get_cmap(cmap)(np.linspace(0,1,len(ax1.lines))))
#colors = [colors(i) for i in np.linspace(0, 1,len(ax1.lines))]
for i,j in enumerate(ax1.lines):
    j.set_color(colors[i])
plt.xlabel('Time t')
plt.ylabel(r'$\dot \theta$')
#plt.ylim(-2,2)
divider = make_axes_locatable(plt.gca())
ax_cb = divider.new_horizontal(size="5%", pad=0.05)
norm = mpl.colors.Normalize(vmin=ts[100], vmax=ts[-25])    
cb1 = mpl.colorbar.ColorbarBase(ax_cb, cmap=mpl.cm.get_cmap(f'{cmap}_r'), orientation='vertical',norm=norm)
#cb1.ax.invert_yaxis()
cb1.set_label('diffusion time (0,1)')
plt.gcf().add_axes(ax_cb)

In [ ]:
i=14 #@param {type:"slider", min:0, max:30, step:1}
vs = -correlate1d(best_reconstructions,np.array([-1,0,1])/2/(ds.T[1]-ds.T[0]),axis=2)
z = ode_datasets.pack(best_reconstructions,(vmap(vmap(vmap(ds.M)))(best_reconstructions)@vs[...,None]).squeeze(-1))
Hs = vmap(vmap(vmap(ds.H)))(z)
fig1 = plt.figure()
ax1 = fig1.add_subplot(111)
data = Hs[100::25,i,:].T
ax1.plot(ds.T_long,data[:],alpha=.6,lw=2)
colors=list(mpl.cm.get_cmap(cmap)(np.linspace(0,1,len(ax1.lines))))
#colors = [colors(i) for i in np.linspace(0, 1,len(ax1.lines))]
for k,j in enumerate(ax1.lines):
    j.set_color(colors[k])
plt.xlabel('Time t')
plt.ylabel(r'Energy')
minn,maxx = data[:,-1].min(),data[:,-1].max()
plt.ylim(minn-2*(maxx-minn),maxx+2*(maxx-minn))
divider = make_axes_locatable(plt.gca())
ax_cb = divider.new_horizontal(size="5%", pad=0.05)
norm = mpl.colors.Normalize(vmin=ts[100], vmax=ts[-25])    
cb1 = mpl.colorbar.ColorbarBase(ax_cb, cmap=mpl.cm.get_cmap(f'{cmap}_r'), orientation='vertical',norm=norm)
#cb1.ax.invert_yaxis()
cb1.set_label('diffusion time (0,1)')
plt.gcf().add_axes(ax_cb)
z0 = z[-1,:,0]
z_gts = vmap(ds.integrate,(0,None),0)(z0,ds.T_long)
ax1.plot(ds.T_long,vmap(vmap(ds.H))(z_gts)[i],color='g',lw=3)

In [ ]:
i=15 # @param {type:"slider", min:0, max:30, step:1}
nn = sample_traj.shape[2]
fft = jnp.abs(np.fft.rfft(sample_traj,axis=2))#[:,:,:nn//2]
freq = np.fft.rfftfreq(sample_traj.shape[2],d=(ds.T[1]-ds.T[0]))#[:nn//2]

fig1 = plt.figure()
ax1 = fig1.add_subplot(111)
data = fft[100::25,i,:,-1].T
ax1.plot(freq,data[:,:],alpha=.6,lw=2)
colors=list(mpl.cm.get_cmap(cmap)(np.linspace(0,1,len(ax1.lines))))
#colors = [colors(i) for i in np.linspace(0, 1,len(ax1.lines))]
for i,j in enumerate(ax1.lines):
    j.set_color(colors[i])
plt.xlabel('Frequency f')
plt.ylabel(r'Fourier spectrum')
plt.yscale('log')
plt.xscale('log')
#plt.ylim(-2,2)
divider = make_axes_locatable(plt.gca())
ax_cb = divider.new_horizontal(size="5%", pad=0.05)
norm = mpl.colors.Normalize(vmin=ts[100], vmax=ts[-25])    
cb1 = mpl.colorbar.ColorbarBase(ax_cb, cmap=mpl.cm.get_cmap(f'{cmap}_r'), orientation='vertical',norm=norm)
#cb1.ax.invert_yaxis()
cb1.set_label('diffusion time (0,1)')
plt.gcf().add_axes(ax_cb)
ax1.plot(freq,jnp.abs(np.fft.rfft(x,axis=1))[::10,:,-1].T,color='blue',alpha=.1);

In [ ]:
i=8 # @param {type:"slider", min:0, max:30, step:1}
nn = best_reconstructions.shape[2]
fft = jnp.abs(np.fft.rfft(best_reconstructions,axis=2))#[:,:,:nn//2]
freq = np.fft.rfftfreq(best_reconstructions.shape[2],d=(ds.T[1]-ds.T[0]))#[:nn//2]

fig1 = plt.figure()
ax1 = fig1.add_subplot(111)
data = fft[100::25,i,:,-1].T
ax1.plot(freq,data[:,:],alpha=.6,lw=2)
colors=list(mpl.cm.get_cmap(cmap)(np.linspace(0,1,len(ax1.lines))))
#colors = [colors(i) for i in np.linspace(0, 1,len(ax1.lines))]
for i,j in enumerate(ax1.lines):
    j.set_color(colors[i])
plt.xlabel('Frequency f')
plt.ylabel(r'Fourier spectrum')
plt.yscale('log')
plt.xscale('log')
#plt.ylim(-2,2)
divider = make_axes_locatable(plt.gca())
ax_cb = divider.new_horizontal(size="5%", pad=0.05)
norm = mpl.colors.Normalize(vmax=ts[100], vmin=ts[-25])    
cb1 = mpl.colorbar.ColorbarBase(ax_cb, cmap=mpl.cm.get_cmap(f'{cmap}_r'), orientation='vertical',norm=norm)
#cb1.ax.invert_yaxis()
cb1.set_label('diffusion time (0,1)')
plt.gcf().add_axes(ax_cb)
ax1.plot(freq,jnp.abs(np.fft.rfft(x,axis=1))[::10,:,-1].T,color='blue',alpha=.1);

In [ ]:

import matplotlib.pyplot as plt
i=24 # @param {type:"slider", min:0, max:30, step:1}
plt.plot(ds.T_long,x[i,:,-1])
plt.plot(ds.T_long,det_samples[i,:,-1])
plt.plot(ds.T_long,stoch_samples[i,:,-1])
plt.xlabel('Time t')
plt.ylabel(r'State')
plt.legend([r'GT',r'Model (ODE)', r'Model (SDE)'])

Test ability to condition model on previous timesteps

In [ ]:
from jax import grad,jit
condition_amount = 13# @param {type:"slider", min:0, max:50, step:1}
mb = x[:30,:]


def inpainting_scores(diffusion,scorefn,observed_values,slc):
  b,n,c = observed_values.shape
  def conditioned_scores(xt,t):
    unflat_xt = xt.reshape(b,-1,c)

    observed_score = diffusion.noise_score(unflat_xt[:,slc],observed_values,t)
    unobserved_score = scorefn(xt,t).reshape(b,-1,c)
    combined_score = unobserved_score.at[:,slc].set(observed_score)
    return combined_score.reshape(-1)
  return conditioned_scores

def inpainting_scores2(diffusion,scorefn,observed_values,slc):
  b,n,c = observed_values.shape
  def conditioned_scores(xt,t):
    unflat_xt = xt.reshape(b,-1,c)

    observed_score = diffusion.noise_score(unflat_xt[:,slc],observed_values,t)
    unobserved_score = scorefn(xt,t).reshape(b,-1,c)
    def constraint(xt):
      one_step_xhat = (xt+diffusion.sigma(t)**2*scorefn(xt,t))/diffusion.scale(t)
      return jnp.sum((one_step_xhat.reshape(b,-1,c)[:,slc]-observed_values)**2)
    #unobserved_score -= grad(constraint)(xt).reshape(unflat_xt.shape)*10/(diff.g2(t)/2)
    unobserved_score -= grad(constraint)(xt).reshape(unflat_xt.shape)*10*diff.scale(t)**2/diff.sigma(t)**2
    combined_score = unobserved_score.at[:,slc].set(observed_score)
    return combined_score.reshape(-1)
  return jit(conditioned_scores)

slc = slice(condition_amount)
conditioned_samples = samplers.stochastic_sample(diff,inpainting_scores2(diff,score_fn,mb[:,slc],slc),key,mb.shape,N=1000,traj=True)


In [ ]:
import matplotlib.pyplot as plt
i=28 # @param {type:"slider", min:0, max:30, step:1}
plt.plot(ds.T_long,conditioned_samples[-600::100,i,:,-1].T,zorder=0,alpha=.2)
plt.plot(ds.T_long,conditioned_samples[-1,i,  :,-1].T,zorder=2)
plt.plot(ds.T_long,x[i,:,-1],label='gt',alpha=1,zorder=99)
plt.plot(ds.T_long[slc],x[i,slc,-1],label='cond',alpha=1,zorder=100,lw=3)

plt.xlabel('Time t')
plt.ylabel(r'State')
plt.ylim(-3,3)
plt.legend()
#plt.legend([r'GT',r'Model'])

In [ ]:
conditioned_sample = samplers.sample(diff,inpainting_scores2(diff,score_fn,mb[:,slc],slc),key,mb.shape)

In [ ]:
from jax import jit,vmap,random

@jit
def rel_err(z1,z2):
  return jnp.abs((jnp.abs(z1-z2)).sum(-1)/(jnp.abs(z1).sum(-1)*jnp.abs(z2).sum(-1)))

gt = x[:30]
for pred in [conditioned_samples[-1],conditioned_sample]:
  clamped_errs = jax.lax.clamp(1e-5,rel_err(pred,gt),np.inf)
  rel_errs = np.exp(jnp.log(clamped_errs).mean(0))
  rel_stds = np.exp(jnp.log(clamped_errs).std(0))
  plt.plot(ds.T_long,rel_errs)
  plt.fill_between(ds.T_long, rel_errs/rel_stds, rel_errs*rel_stds,alpha=.1)

plt.plot()
plt.yscale('log')
plt.xlabel('Time')
plt.ylabel('Prediction Error')
plt.legend(['SDE completion','ODE completion'])

In [ ]:
i=2 # @param {type:"slider", min:0, max:29, step:1}
plt.plot(ds.T_long,x[i,:,-1])
plt.plot(ds.T_long[slc],x[i,slc,-1],lw=3)
plt.plot(ds.T_long,conditioned_sample[i,:,-1])
plt.xlabel('Time t')
plt.ylabel(r'State')
plt.legend([r'GT','Conditioning',r'Model'])
plt.ylim(-3,3)

In [ ]:
import jax 
jax.config.update('jax_default_matmul_precision', 'float32')

Energy Conditioning

In [ ]:

from jax.scipy.stats.multivariate_normal import logpdf as normal_logpdf
from jax import grad,jit,vmap
key = jax.random.PRNGKey(38)
mb = x[:30,:]
b,_,c = mb.shape

def H(q):
  # takes as input q shaped (n,c)
  #v = -correlate1d(one_step_xhat,np.array([-1,0,1])/2/(ds.T[1]-ds.T[0]),axis=1)
  v = -(q[:-2]-q[2:])/(2*(ds.T[1]-ds.T[0]))
  z = ode_datasets.pack(q[1:-1],(vmap(ds.M)(q[1:-1])@v[...,None]).squeeze(-1))
  Hs = vmap(ds.H)(z)
  return Hs

def C(q):
  # takes as input q shaped (n,c)
  Hs = H(q)
  return Hs-Hs.mean(axis=0,keepdims=True)

data_std = thetas.std()

def energy_conditioned_scores(diffusion,scorefn):
  def conditioned_scores(xt,t):
    # if not t.shape:
    #   t=jnp.ones(b)*t
    unflat_xt = xt.reshape(b,-1,c)
    n = unflat_xt.shape[1]
    unobserved_score = scorefn(xt,t).reshape(b,-1,c)
    # def constraint(xt):
    #   qhat = (xt+diffusion.sigma(t)**2*scorefn(xt,t))/diffusion.scale(t)
    #   qhat = qhat.reshape(b,-1,c)
    #   return jnp.sum(jnp.square(C(qhat)))
    def xhat(xt):
      score_xhat = (xt+diffusion.sigma(t)**2*scorefn(xt,t))/diffusion.scale(t)
      #print('score',score_xhat)
      limiting_xhat = (xt/(1+diffusion.sigma(t)**2/data_std**2))/diffusion.scale(t)
      #print('limiting',limiting_xhat)
      tau = jnp.maximum(2*(t-1)+1.,0.)
      #tau = jnp.minimum(2*t,1.0)
      #ratio = (limiting_xhat/score_xhat)

      blended = score_xhat#*jnp.sign(ratio)*jnp.abs(ratio)**tau
      #print('blended',blended)
      return blended

    # def Sigma(xt):
    #   return (diff.sigma(t)/diff.scale(t))**2/(1+diffusion.sigma(t)**2/data_std**2)
      # #b,n,c = xt.shape
      # xt = xt.reshape(b,-1)
      # xh = vmap(xhat)(xt).reshape(b,n,c)
      # DC = vmap(jax.jacrev(C))(xh).reshape(b,-1,n*c)
      # #DC2 = vmap(jax.jacrev(lambda x: C(xhat(x[None])[0].reshape(n,c))))(xt)*diffusion.scale(t)
      # #sig = (DC@jnp.swapaxes(DC,-1,-2))/(1+diffusion.sigma(t)**2/data_std**2)
      # scale = jnp.trace(DC@jnp.swapaxes(DC,-1,-2),axis1=-1,axis2=-2)/DC.shape[1]/(1+diffusion.sigma(t)**2/data_std**2)
      # return scale*(diff.sigma(t)/diff.scale(t))**2
    sigma = ((diff.sigma(t)/diff.scale(t))**2)#/(1+diffusion.sigma(t)**2/data_std**2) 
    def log_likelihood(xt):
      qhat = xhat(xt).reshape(b,-1,c)
      value = vmap(C)(qhat)
      return -(value*value).sum()/(2.*sigma)
      #return normal_logpdf(value,jnp.zeros_like(value),Sigma(xt)).sum()
    unobserved_score += .5*grad(log_likelihood)(xt).reshape(unflat_xt.shape)
    return unobserved_score.reshape(-1)
  return jit(conditioned_scores)



In [ ]:
energy_samples_det = samplers.sample(diff,energy_conditioned_scores(diff,score_fn),key,mb.shape)
print(f'ODE performance {pmetric(energy_samples_det)[0]}')

In [ ]:

energy_samples_stoch = samplers.stochastic_sample(diff,energy_conditioned_scores(diff,score_fn),key,mb.shape,N=100,traj=False)
print(f'SDE performance {pmetric(energy_samples_stoch)[0]}')

In [ ]:
print(f'SDE performance {pmetric(energy_samples_stoch)[0]}')

In [ ]:
from scipy.ndimage import correlate1d
energy_samples_traj = samplers.stochastic_sample(diff,energy_conditioned_scores(diff,score_fn),key,mb.shape,N=100,traj=True)
n=energy_samples_traj.shape[0]+1
ts = (.5+jnp.arange(n)[::-1])[:-1]/n
scores = vmap(score_fn)(energy_samples_traj.reshape(len(ts),-1),ts).reshape(energy_samples_traj.shape)
best_reconstructions = (energy_samples_traj+diff.sigma(ts)[:,None,None,None]**2*scores)/diff.scale(ts)[:,None,None,None]
vs = -correlate1d(best_reconstructions,np.array([-1,0,1])/2/(ds.T[1]-ds.T[0]),axis=2)
z = ode_datasets.pack(best_reconstructions,(vmap(vmap(vmap(ds.M)))(best_reconstructions)@vs[...,None]).squeeze(-1))
Hs = vmap(vmap(vmap(ds.H)))(z)
kstart=10
z0 = z[-1,:,kstart]
#print(vmap(ds.H)(z0)[i])
z_gts = vmap(ds.integrate,(0,None),0)(z0,ds.T_long[kstart:])

In [ ]:
jnp.any(jnp.isnan(energy_samples_traj),(-2,-1)).mean(-1)

In [ ]:
import matplotlib.pyplot as plt

import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable
cmap='inferno'
i=1 #@param {type:"slider", min:0, max:30, step:1}

fig1 = plt.figure()
ax1 = fig1.add_subplot(111)
data = Hs[10::5,i,:].T
#print(data[:,-1])
ax1.plot(ds.T_long,data[::-1],alpha=.6,lw=2)
colors=list(mpl.cm.get_cmap(cmap)(np.linspace(0,1,len(ax1.lines))))
#colors = [colors(i) for i in np.linspace(0, 1,len(ax1.lines))]
for k,j in enumerate(ax1.lines):
    j.set_color(colors[k])
plt.xlabel('Time t')
plt.ylabel(r'Energy')
minn,maxx = data[:,-1].min(),data[:,-1].max()
plt.ylim(minn-2*(maxx-minn),maxx+2*(maxx-minn))
divider = make_axes_locatable(plt.gca())
ax_cb = divider.new_horizontal(size="5%", pad=0.05)
norm = mpl.colors.Normalize(vmin=ts[100], vmax=ts[-25])    
cb1 = mpl.colorbar.ColorbarBase(ax_cb, cmap=mpl.cm.get_cmap(f'{cmap}_r'), orientation='vertical',norm=norm)
#cb1.ax.invert_yaxis()
cb1.set_label('diffusion time (0,1)')
plt.gcf().add_axes(ax_cb)

ax1.plot(ds.T_long[kstart:],vmap(vmap(ds.H))(z_gts)[i],color='g',lw=3)

In [ ]:
data_std

In [ ]:
diffusion=diff
scorefn=score_fn
N=1000
timesteps = (.5+np.arange(N)[::-1])/N


# def xhat(xt,t):
#   return (xt+diffusion.sigma(t)**2*scorefn(xt,t))/diffusion.scale(t)
def xhat(xt,t):
    score_xhat = (xt+diffusion.sigma(t)**2*scorefn(xt,t))/diffusion.scale(t)
    #print('score',score_xhat)
    limiting_xhat = (xt/(1+diffusion.sigma(t)**2/data_std**2))/diffusion.scale(t)
    #print('limiting',limiting_xhat)
    tau = jnp.minimum(3*t,1.0)
    ratio = (limiting_xhat/score_xhat)

    blended = score_xhat*jnp.sign(ratio)*jnp.abs(ratio)**tau
    #print('blended',blended)
    return blended
def Sigma(xt,t):
  b,n,c = xt.shape
  xt = xt.reshape(b,-1)
  xh = vmap(xhat)(xt,t).reshape(b,n,c)
  DC = vmap(jax.jacrev(C))(xh).reshape(b,-1,n*c)
  DC2 = vmap(jax.jacrev(lambda x,t: C(xhat(x[None],t)[0].reshape(n,c))))(xt,t)
  sig = (DC@jnp.swapaxes(DC2,-1,-2))*diffusion.scale(t)[:,None,None]
  reg = 1e-5*jnp.eye(sig.shape[-1])[None]/(1+diffusion.sigma(t)**2/data_std**2)[:,None,None]
  #eigs,V = jnp
  #rint(jnp.linalg.eigh(sig)[0])
  return ((sig + jnp.swapaxes(sig,-1,-2))/2. + reg)#*(1+diffusion.sigma(t)**2/data_std**2)[:,None,None]#*(diff.scale(t)**2/diff.sigma(t)**2)

In [ ]:
ts = (timesteps[:-1])[::100]
S = Sigma(sample_traj[::100,0],ts)

In [ ]:
sample_traj[0,0].shape

In [ ]:
xhat(sample_traj[0,0].reshape(-1),ts[0]);

In [ ]:
t  =timesteps[::200]
xt = sample_traj[::200,0]
b,n,c = xt.shape
xt = xt.reshape(b,-1)
xh = vmap(xhat)(xt,t)
DC = vmap(jax.jacfwd(lambda x: C(x.reshape(n,c))))(xh)#.reshape(b,-1,n*c)
DC2 = vmap(jax.jacfwd(lambda x,t: C(xhat(x,t).reshape(n,c))))(xt,t)
SS = vmap(jax.jacfwd(xhat))(xt,t)
sig = (DC@jnp.swapaxes(DC2,-1,-2))*diffusion.scale(t)[:,None,None]
sig = (sig+jnp.swapaxes(sig,-1,-2))/2
eigs,V = jnp.linalg.eigh(sig)
cliplevel = 1e-4*eigs[:,-1]
eigs = jnp.where(eigs<cliplevel[:,None],cliplevel[:,None],eigs) 
sig = V@vmap(jnp.diag)(eigs)@jnp.swapaxes(V,-1,-2)

In [ ]:
eigs[0,0]

In [ ]:
rel_err(V@vmap(jnp.diag)(eigs)@jnp.swapaxes(V,-1,-2),sig)

In [ ]:

w = jnp.linalg.eigh(S)[0]

In [ ]:
w[0]

In [ ]:
#print(w)
for i,ti in enumerate(ts):
  plt.plot(w[i],label=str(ti))
plt.legend()
plt.ylabel(r'$\lambda_i$')
plt.xlabel(r'$i$')
#plt.ylim(1e-3,20)
#plt.ylim(-2,10)
plt.yscale('log')

In [ ]:

w = jnp.linalg.eigh(sig)[0]
#print(w)
for i,ti in enumerate(t):
  plt.plot(w[i],label=str(ti))
plt.legend()
plt.ylabel(r'$\lambda_i$')
plt.xlabel(r'$i$')
# plt.ylim(1e-3,20)
plt.yscale('log')

In [ ]:
xh[0]

In [ ]:
w[0]

In [ ]:
from functools import partial
vnorm = vmap(partial(jnp.linalg.norm,ord=2))
def rel_err(A,B):
  return vnorm(A-B)/(vnorm(A)+vnorm(B))

In [ ]:
sig.shape

In [ ]:
#jnp.abs(SS@jnp.swapaxes(DC,-1,-2)-jnp.swapaxes(DC2,-1,-2)).mean()
rel_err(SS@jnp.swapaxes(DC,-1,-2),jnp.swapaxes(DC2,-1,-2))

In [ ]:
#jnp.abs(DC@SS-DC2).mean()/jnp.abs(DC2).mean()
rel_err(DC@SS,DC2)

In [ ]:
#jnp.abs(SS-jnp.swapaxes(SS,-1,-2)).mean()/jnp.abs(SS).mean()
rel_err(SS,jnp.swapaxes(SS,-1,-2))

In [ ]:
rel_err(sig,jnp.swapaxes(sig,-1,-2))

In [ ]:
sig2 = (DC@SS)@jnp.swapaxes(DC,-1,-2)*diffusion.scale(t)[:,None,None]
#jnp.abs(sig2-sig).mean()/jnp.abs(sig).mean()
rel_err(sig2,sig)

In [ ]:
jnp.abs(sig2-jnp.swapaxes(sig,-1,-2)).mean()/jnp.abs(sig2).mean()

In [ ]:

jnp.abs(sig-jnp.swapaxes(sig,-1,-2)).mean()

In [ ]:
jnp.linalg.norm(jnp.eye(20),ord=2)

In [ ]:

rel_err(sig2,jnp.swapaxes(sig,-1,-2))

In [ ]:
from jax.config import config
config.update("jax_enable_x64", True)

In [ ]:
import jax 
jax.config.update('jax_default_matmul_precision', 'float32')

In [ ]:
f = lambda x,t: C(xhat(x,t).reshape(n,c))
DC2 = vmap(jax.jacfwd(f))(xt,t)
DC2r = vmap(jax.jacrev(f))(xt,t)
jnp.abs(DC2-DC2r).mean()/jnp.abs(DC2).mean()

In [ ]:
print('FWD',DC2[1,:3,:3])
print('BWD',DC2r[1,:3,:3])

In [ ]:
DC.shape

In [ ]:
vmap(xhat)(xt,t).shape

In [ ]:
xt.shape

In [ ]:
jnp.abs(S[3]-jnp.swapaxes(S[3],-1,-2)).mean()

In [ ]:
eigs,V = jnp.linalg.eigh(S)


In [ ]:
jnp.abs(V[1]@jnp.diag(eigs[1])@V[1].T -(S[1]+S[1].T)/2).mean()

In [ ]:
w = jnp.linalg.eigh(S)[0]
for i,t in enumerate(ts):
  plt.plot(w[i],label=str(t))
plt.legend()
plt.ylabel(r'$\lambda_i$')
plt.xlabel(r'$i$')
#plt.ylim(-2,10)
plt.yscale('log')

In [ ]:
%debug

In [ ]:
energy_samples_stoch = samplers.stochastic_sample(diff,energy_conditioned_scores(diff,score_fn),key,mb.shape,N=1000,traj=False)
print(f'SDE performance {pmetric(energy_samples_stoch)[0]}')

In [ ]:
%debug

In [ ]:
energy_samples_det = samplers.sample(diff,energy_conditioned_scores(diff,score_fn),key,mb.shape)
print(f'ODE performance {pmetric(energy_samples_det)[0]}')

In [ ]:
from scipy.ndimage import correlate1d
energy_samples_traj = samplers.stochastic_sample(diff,energy_conditioned_scores(diff,score_fn),key,mb.shape,N=1000,traj=True)
n=energy_samples_traj.shape[0]+1
ts = (.5+jnp.arange(n)[::-1])[:-1]/n
scores = vmap(score_fn)(energy_samples_traj.reshape(len(ts),-1),ts).reshape(energy_samples_traj.shape)
best_reconstructions = (energy_samples_traj+diff.sigma(ts)[:,None,None,None]**2*scores)/diff.scale(ts)[:,None,None,None]
vs = -correlate1d(best_reconstructions,np.array([-1,0,1])/2/(ds.T[1]-ds.T[0]),axis=2)
z = ode_datasets.pack(best_reconstructions,(vmap(vmap(vmap(ds.M)))(best_reconstructions)@vs[...,None]).squeeze(-1))
Hs = vmap(vmap(vmap(ds.H)))(z)
kstart=10
z0 = z[-1,:,kstart]
#print(vmap(ds.H)(z0)[i])
z_gts = vmap(ds.integrate,(0,None),0)(z0,ds.T_long[kstart:])

In [ ]:
import matplotlib.pyplot as plt

import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable
cmap='inferno'
i=5 #@param {type:"slider", min:0, max:30, step:1}

fig1 = plt.figure()
ax1 = fig1.add_subplot(111)
data = Hs[100::25,i,:].T
#print(data[:,-1])
ax1.plot(ds.T_long,data[::-1],alpha=.6,lw=2)
colors=list(mpl.cm.get_cmap(cmap)(np.linspace(0,1,len(ax1.lines))))
#colors = [colors(i) for i in np.linspace(0, 1,len(ax1.lines))]
for k,j in enumerate(ax1.lines):
    j.set_color(colors[k])
plt.xlabel('Time t')
plt.ylabel(r'Energy')
minn,maxx = data[:,-1].min(),data[:,-1].max()
plt.ylim(minn-2*(maxx-minn),maxx+2*(maxx-minn))
divider = make_axes_locatable(plt.gca())
ax_cb = divider.new_horizontal(size="5%", pad=0.05)
norm = mpl.colors.Normalize(vmin=ts[100], vmax=ts[-25])    
cb1 = mpl.colorbar.ColorbarBase(ax_cb, cmap=mpl.cm.get_cmap(f'{cmap}_r'), orientation='vertical',norm=norm)
#cb1.ax.invert_yaxis()
cb1.set_label('diffusion time (0,1)')
plt.gcf().add_axes(ax_cb)

ax1.plot(ds.T_long[kstart:],vmap(vmap(ds.H))(z_gts)[i],color='g',lw=3)

In [ ]:
@jit
def rel_err(x,y):
  return  jnp.abs(x-y).sum(-1)/(jnp.abs(x).sum(-1)+jnp.abs(y).sum(-1))


kstart=10
@jit
def log_prediction_metric(qs):
  k=kstart
  q = qs[k:]
  v = -(q[:-2]-q[2:])/(2*(ds.T[1]-ds.T[0]))
  #print(vmap(ds.M)(q[1:-1]).shape,v.shape)
  z = ode_datasets.pack(q[1:-1],(vmap(ds.M)(q[1:-1])@v[...,None]).squeeze(-1))
  T = ds.T_long[k+1:-1]
  z_gt = ds.integrate(z[0],T)
  return jnp.log(rel_err(z,z_gt)[1:len(T)//3]).mean()

@jit
def pmetric(qs):
  log_metric = vmap(log_prediction_metric)(qs)
  return jnp.exp(log_metric.mean()),jnp.exp(log_metric.std()/jnp.sqrt(log_metric.shape[0]))

Unconditional Prediction quality

In [ ]:
stoch_samples = samplers.stochastic_sample(diff,score_fn,key,x[:30].shape,N=2000,traj=False)
det_samples = samplers.sample(diff,score_fn,key,x[:30].shape)
print(f'ODE performance {pmetric(det_samples)[0]}')
print(f'SDE performance {pmetric(stoch_samples)[0]}')

In [ ]:
key = random.PRNGKey(45)
#s=s2#,history = samplers.stochastic_sampler(denoiser,params,key,(32,)+data.shape[1:],N=500,smin=sigma_min,smax=sigma_max)
s = stoch_samples

k = 5
q = s[:,k:]
v = -(q[:,:-2]-q[:,2:])/(2*(ds.T[1]-ds.T[0]))
z = ode_datasets.pack(q[:,1:-1],(vmap(vmap(ds.M))(q[:,1:-1])@v[...,None]).squeeze(-1))
T = ds.T_long[k+1:-1]
z0 = z[:,0]
z_gts = vmap(ds.integrate,(0,None),0)(z0,T)
z_pert = vmap(ds.integrate,(0,None),0)(z0+1e-3*np.random.randn(*z0.shape),T)
z_random = vmap(ds.integrate,(0,None),0)(ds.sample_initial_conditions(z0.shape[0]),T)


In [ ]:
for pred in [z,z_pert,z_random]:
  clamped_errs = jax.lax.clamp(1e-3,rel_err(pred,z_gts),np.inf)
  rel_errs = np.exp(jnp.log(clamped_errs).mean(0))
  rel_stds = np.exp(jnp.log(clamped_errs).std(0))
  plt.plot(T,rel_errs)
  plt.fill_between(T, rel_errs/rel_stds, rel_errs*rel_stds,alpha=.1)

plt.plot()
plt.yscale('log')
plt.xlabel('Time')
plt.ylabel('Prediction Error')
plt.legend(['Diffusion Model Rollout','1e-3 Perturbed GT','Random Init'])

In [ ]:
H_gts = vmap(vmap(ds.H))(z_gts)
for pred in [z,z_pert,z_random]:
  Hs = vmap(vmap(ds.H))(pred)
  clamped_errs = jax.lax.clamp(1e-3,jnp.abs(Hs-H_gts)/jnp.abs(Hs*H_gts),np.inf)
  rel_errs = np.exp(jnp.log(clamped_errs).mean(0))
  rel_stds = np.exp(jnp.log(clamped_errs).std(0))
  plt.plot(T,rel_errs)
  plt.fill_between(T, rel_errs/rel_stds, rel_errs*rel_stds,alpha=.1)

plt.plot()
plt.yscale('log')
plt.xlabel('Time')
plt.ylabel('Energy Error')
plt.legend(['Diffusion Model Rollout','1e-3 Perturbed GT','Random Init'])

Compared trajectories

In [ ]:
for i in range(10):
  fig = plt.figure()
  ax = fig.add_subplot(1, 1, 1)
  line1, = ax.plot(T,z_gts[i,:,0])
  line2, = ax.plot(T,z[i,:,0])
  line3, = ax.plot(T,z_pert[i,:,0])
  plt.xlabel('Time t')
  plt.ylabel(r'State')
  plt.legend(['gt','model','pert'])

In [ ]:
for i in range(10):
  fig = plt.figure()
  ax = fig.add_subplot(1, 1, 1)
  line1, = ax.plot(T,z_gts[i,:,0])
  line2, = ax.plot(T,z[i,:,0])
  line3, = ax.plot(T,z_gts[i,:,-1])
  line5, = ax.plot(T,z[i,:,-1])
  plt.xlabel('Time t')
  plt.ylabel(r'State')
  plt.legend([r'$\theta_0$ gt',r'$\theta_0$ model',r'v gt', r'v model'])

In [ ]:
metric_vals =[]
metric_stds = []
Ns = [25,50,100,200,500,1000,2000]
for N in Ns:
  s = samplers.stochastic_sample(diff,score_fn,key,x[:30].shape,N=N)
  mean,std = pmetric(s)
  metric_vals.append(mean)
  metric_stds.append(std)
metric_vals = np.array(metric_vals)
metric_stds = np.array(metric_stds)

plt.plot(Ns,metric_vals)
plt.fill_between(Ns, metric_vals/metric_stds, metric_vals*metric_stds,alpha=.3)
plt.xlabel('Sampler steps')
plt.ylabel('Pmetric value')
plt.xscale('log')

In [ ]:
ds2 = ode_datasets.NPendulum(N=50,n=2,dt=.002)
thetas2,vs2 = ode_datasets.unpack(ds2.Zs)
test_x2 = thetas2
#test_x2 = -correlate1d(test_x2,np.array([-1,0,1])/2/(ds.T[1]-ds.T[0]),axis=1)
freq = np.fft.rfftfreq(test_x2.shape[1],d=(ds2.T[1]-ds2.T[0]))
ffts  =jnp.abs(np.fft.rfft(test_x2,axis=1))
avg_freq  =jnp.exp(jnp.log(ffts)[:,:,-1]).mean(0)
plt.plot(freq,avg_freq.T,color='blue',label=r'2 Pendulum $\theta$');
plt.plot(freq,100/freq,label=r'1/f')
plt.plot(freq,30/freq**2,label=r'1/$f^2$')
plt.plot(freq,10/freq**3,label=r'1/$f^3$')
#plt.plot(freq,np.exp(-10*freq**2),label=r'$e^{-f^2}$')
plt.plot(freq,ffts[:,:,-1].T,color='y',alpha=.2);
plt.yscale('log')
plt.xscale('log')
plt.ylim(1e-3,1e5)
plt.xlabel('Frequency f')
plt.ylabel(r'Fourier spectrum')
plt.legend()